<a href="https://colab.research.google.com/github/ericyoc/opt_LRU_cache_perf/blob/main/opt_LRU_cache_perf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import OrderedDict
from tabulate import tabulate
import random

In [11]:
class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache = OrderedDict()

    def get(self, key):
        if key not in self.cache:
            return -1

        self.cache.move_to_end(key)
        return self.cache[key]

    def put(self, key, value):
        if key in self.cache:
            self.cache.move_to_end(key)
        else:
            if len(self.cache) == self.capacity:
                self.cache.popitem(last=False)

            self.cache[key] = value

class FIFOCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache = OrderedDict()

    def get(self, key):
        if key not in self.cache:
            return -1

        return self.cache[key]

    def put(self, key, value):
        if key in self.cache:
            self.cache.pop(key)

        if len(self.cache) == self.capacity:
            self.cache.popitem(last=False)

        self.cache[key] = value

class ENRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache = OrderedDict()
        self.top_layer_size = capacity // 8
        self.middle_layer_size = capacity // 4
        self.bottom_layer_size = (5 * capacity) // 8
        self.top_layer = OrderedDict()
        self.middle_layer = OrderedDict()
        self.bottom_layer = OrderedDict()
        self.reference_bits = {}
        self.dirty_bits = {}

    def get(self, key):
        if key not in self.cache:
            return -1

        if key in self.top_layer:
            self.top_layer.move_to_end(key)
            self.reference_bits[key] = 1
        elif key in self.middle_layer:
            self.middle_layer.move_to_end(key)
            block = self.middle_layer.popitem(last=False)
            self.top_layer[block[0]] = block[1]
            self.reference_bits[block[0]] = 1
            if len(self.top_layer) > self.top_layer_size:
                evicted_block = self.find_block_to_replace()
                self.top_layer.pop(evicted_block)
                self.reference_bits.pop(evicted_block)
                self.dirty_bits.pop(evicted_block)
        elif key in self.bottom_layer:
            self.bottom_layer.move_to_end(key)
            block = self.bottom_layer.popitem(last=False)
            self.middle_layer[block[0]] = block[1]
            if len(self.middle_layer) > self.middle_layer_size:
                evicted_block, _ = self.middle_layer.popitem(last=False)
                self.top_layer[evicted_block] = self.cache[evicted_block]
                self.reference_bits[evicted_block] = 1
                if len(self.top_layer) > self.top_layer_size:
                    evicted_block = self.find_block_to_replace()
                    self.top_layer.pop(evicted_block)
                    self.reference_bits.pop(evicted_block)
                    self.dirty_bits.pop(evicted_block)

        return self.cache[key]

    def put(self, key, value):
        if key in self.cache:
            self.cache[key] = value
            self.dirty_bits[key] = 1
            if key in self.top_layer:
                self.top_layer.move_to_end(key)
            elif key in self.middle_layer:
                self.middle_layer.move_to_end(key)
            else:
                self.bottom_layer.move_to_end(key)
        else:
            self.cache[key] = value
            self.bottom_layer[key] = value
            self.reference_bits[key] = 0
            self.dirty_bits[key] = 1
            if len(self.bottom_layer) > self.bottom_layer_size:
                evicted_block, _ = self.bottom_layer.popitem(last=False)
                self.middle_layer[evicted_block] = self.cache[evicted_block]
                if len(self.middle_layer) > self.middle_layer_size:
                    evicted_block, _ = self.middle_layer.popitem(last=False)
                    self.top_layer[evicted_block] = self.cache[evicted_block]
                    self.reference_bits[evicted_block] = 1
                    if len(self.top_layer) > self.top_layer_size:
                        evicted_block = self.find_block_to_replace()
                        self.top_layer.pop(evicted_block)
                        self.reference_bits.pop(evicted_block)
                        self.dirty_bits.pop(evicted_block)

    def find_block_to_replace(self):
        for block in self.top_layer:
            if self.reference_bits[block] == 0 and self.dirty_bits[block] == 0:
                return block
        for block in self.top_layer:
            if self.reference_bits[block] == 0 and self.dirty_bits[block] == 1:
                return block
        for block in self.top_layer:
            if self.reference_bits[block] == 1 and self.dirty_bits[block] == 0:
                return block
        return next(iter(self.top_layer))
def simulate_cache(cache_class, capacity, access_pattern):
    cache = cache_class(capacity)
    hits = 0
    misses = 0

    for key in access_pattern:
        if cache.get(key) != -1:
            hits += 1
        else:
            misses += 1
            cache.put(key, None)

    return hits / (hits + misses) if (hits + misses) > 0 else 0

def generate_access_pattern(access_pattern_size, unique_keys, skew_factor):
    access_pattern = []
    for _ in range(access_pattern_size):
        if random.random() < skew_factor:
            key = random.randint(0, int(unique_keys * 0.2))  # 20% of keys are frequently accessed
        else:
            key = random.randint(int(unique_keys * 0.2) + 1, unique_keys - 1)
        access_pattern.append(key)
    return access_pattern

def main():
    cache_capacity = 50
    access_pattern_size = 10000
    unique_keys = 500

    # Generate access patterns
    default_access_pattern = generate_access_pattern(access_pattern_size, unique_keys, skew_factor=0.2)
    conditional_access_pattern = generate_access_pattern(access_pattern_size, unique_keys, skew_factor=0.8)

    cache_algorithms = [
        {
            "name": "Least Recently Used (LRU)",
            "class": LRUCache,
            "strengths": "Keeps recently accessed items in the cache",
            "weaknesses": "Can evict a frequently accessed item if it hasn't been accessed recently",
            "importance": "Widely used in various caching systems",
            "performance_default": simulate_cache(LRUCache, cache_capacity, default_access_pattern),
            "performance_conditional": simulate_cache(LRUCache, cache_capacity, conditional_access_pattern)
        },
        {
            "name": "First In First Out (FIFO)",
            "class": FIFOCache,
            "strengths": "Simple and easy to implement",
            "weaknesses": "Doesn't consider access patterns, can evict frequently accessed items",
            "importance": "Used in situations where access patterns are unknown or unpredictable",
            "performance_default": simulate_cache(FIFOCache, cache_capacity, default_access_pattern),
            "performance_conditional": simulate_cache(FIFOCache, cache_capacity, conditional_access_pattern)
        },
        {
            "name": "Enhanced Not Recently Used (ENRU)",
            "class": ENRUCache,
            "strengths": "Combines the benefits of LRU and FIFO, keeps frequently accessed items in LRU list and less frequently accessed items in NRU list",
            "weaknesses": "More complex implementation than LRU and FIFO",
            "importance": "Designed for scenarios with conditional access patterns and limited cache capacity",
            "performance_default": simulate_cache(ENRUCache, cache_capacity, default_access_pattern),
            "performance_conditional": simulate_cache(ENRUCache, cache_capacity, conditional_access_pattern)
        }
    ]

    print(tabulate(
        [
            [
                algo["name"],
                algo["strengths"],
                algo["weaknesses"],
                algo["importance"],
                f"{algo['performance_default']:.3f}",
                f"{algo['performance_conditional']:.3f}"
            ]
            for algo in cache_algorithms
        ],
        headers=["Algorithm", "Strengths", "Weaknesses", "Importance", "Performance (Default)", "Performance (Conditional)"],
        tablefmt="grid"
    ))

In [12]:
if __name__ == "__main__":
    main()

+-----------------------------------+-----------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------+------------------------------------------------------------------------------------+-------------------------+-----------------------------+
| Algorithm                         | Strengths                                                                                                                         | Weaknesses                                                               | Importance                                                                         |   Performance (Default) |   Performance (Conditional) |
+===================================+===================================================================================================================================+===========================================================